<a href="https://colab.research.google.com/github/anupojuharshita/GEN_AI-project/blob/main/curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import files

def detect_curve(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if out is None:
            height, width, _ = frame.shape
            out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian Blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # Detect edges using Canny
        edges = cv2.Canny(blurred, 50, 150)

        # Define region of interest
        height, width = edges.shape
        mask = np.zeros_like(edges)
        polygon = np.array([
            [(0, height), (width, height), (width // 2, height // 2)]
        ])
        cv2.fillPoly(mask, polygon, 255)
        roi = cv2.bitwise_and(edges, mask)

        # Hough Transform to detect lines
        lines = cv2.HoughLinesP(roi, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        out.write(frame)

    cap.release()
    out.release()

    print(f"Processed video saved at: {output_path}")

# Upload video file
uploaded = files.upload()
input_video = list(uploaded.keys())[0]
output_video = "/content/curve_detected.mp4"

detect_curve(input_video, output_video)


Saving curve2.mp4 to curve2.mp4
Processed video saved at: /content/curve_detected.mp4


In [9]:
import cv2
import numpy as np
import os
from google.colab import files

def region_of_interest(img):
    height, width = img.shape
    mask = np.zeros_like(img)
    polygon = np.array([
        [(int(width * 0.1), height), (int(width * 0.9), height),
         (int(width * 0.6), int(height * 0.5)), (int(width * 0.4), int(height * 0.5))]
    ])
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

def draw_lanes(frame, lines):
    if lines is None:
        return frame

    left_lines = []
    right_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        slope = (y2 - y1) / (x2 - x1 + 1e-6)  # Avoid division by zero
        if slope < 0:
            left_lines.append(line[0])
        else:
            right_lines.append(line[0])

    lane_image = np.zeros_like(frame)
    for line in left_lines + right_lines:
        x1, y1, x2, y2 = line
        cv2.line(lane_image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    return cv2.addWeighted(frame, 0.8, lane_image, 1, 1)

def detect_curve(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if out is None:
            height, width, _ = frame.shape
            out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        roi = region_of_interest(edges)

        lines = cv2.HoughLinesP(roi, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=50)
        frame = draw_lanes(frame, lines)

        out.write(frame)

    cap.release()
    out.release()
    print(f"Processed video saved at: {output_path}")

uploaded = files.upload()
input_video = list(uploaded.keys())[0]
output_video = "/content/curve_detected.mp4"

detect_curve(input_video, output_video)

from google.colab import files
files.download(output_video)

Saving curve2.mp4 to curve2 (1).mp4
Processed video saved at: /content/curve_detected.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import cv2
import numpy as np
import os
from google.colab import files

def detect_curve(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 20.0

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Create a copy of the frame for visualization
        result = frame.copy()

        # Convert to HSV for better color detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Yellow lane detection (center line)
        lower_yellow = np.array([15, 80, 80])
        upper_yellow = np.array([35, 255, 255])
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

        # White lane detection (side line)
        lower_white = np.array([0, 0, 180])
        upper_white = np.array([180, 30, 255])
        white_mask = cv2.inRange(hsv, lower_white, upper_white)

        # Create a region of interest mask
        mask = np.zeros_like(yellow_mask)
        height, width = yellow_mask.shape
        roi_vertices = np.array([
            [(int(width * 0.1), height),
             (int(width * 0.9), height),
             (int(width * 0.6), int(height * 0.5)),
             (int(width * 0.4), int(height * 0.5))]
        ], dtype=np.int32)
        cv2.fillPoly(mask, [roi_vertices], 255)

        # Apply ROI mask to both yellow and white masks
        yellow_roi = cv2.bitwise_and(yellow_mask, mask)
        white_roi = cv2.bitwise_and(white_mask, mask)

        # Apply some morphological operations to improve lane detection
        kernel = np.ones((5, 5), np.uint8)
        yellow_roi = cv2.dilate(yellow_roi, kernel, iterations=1)
        white_roi = cv2.dilate(white_roi, kernel, iterations=1)

        # Find contours for both yellow and white lanes
        yellow_contours, _ = cv2.findContours(yellow_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        white_contours, _ = cv2.findContours(white_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours by area to remove noise
        min_area = 100
        filtered_yellow = [c for c in yellow_contours if cv2.contourArea(c) > min_area]
        filtered_white = [c for c in white_contours if cv2.contourArea(c) > min_area]

        # Draw lane lines
        cv2.drawContours(result, filtered_yellow, -1, (0, 255, 0), 3)  # Green color for yellow lane
        cv2.drawContours(result, filtered_white, -1, (0, 255, 0), 3)   # Green color for white lane

        # Add text indicating lane detection
        cv2.putText(result, "Lane Detection", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                   1, (0, 255, 0), 2, cv2.LINE_AA)

        # Write frame to output video
        out.write(result)

    # Release resources
    cap.release()
    out.release()
    print(f"Processed video saved at: {output_path}")

# Upload video
print("Please upload your video file:")
uploaded = files.upload()
input_video = list(uploaded.keys())[0]
output_video = "/content/curve_detected.mp4"

# Process video
detect_curve(input_video, output_video)

# Download processed video
files.download(output_video)

Please upload your video file:


Saving curve2.mp4 to curve2 (1).mp4
Processed video saved at: /content/curve_detected.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>